In [4]:
import pandas as pd
from pathlib import Path

output_file = './tsv/Second_Phase_Dataset.tsv'
folder_path = './AICUP/Second_Phase_Dataset/Second_Phase_Dataset/Second_Phase_Text_Dataset'

def extract_number_from_filename(file_path):
    return int(''.join(filter(str.isdigit, file_path.stem)))

# 創建一個空的 DataFrame
df = pd.DataFrame(columns=['fid', 'idx', 'context', 'label'])
data_list = []

file_list = sorted(Path(folder_path).rglob('*.txt'), key=extract_number_from_filename)

for file_path in file_list:
    with open(file_path, 'r', encoding='utf-8') as file:
        char_position = 0
        for idx, line in enumerate(file, start=1):
            if line.strip():
                data_list.append({'fid': file_path.stem, 'idx': char_position, 'context': line.strip().replace('\t',' '), 'label': 'PHI: NULL'})
                char_position += len(line.rstrip('\n'))

df = pd.DataFrame(data_list)
df.reset_index(drop=True, inplace=True)


In [ ]:
fid_set = set(df['fid'].values)
print(fid_set)
fid = '1534'
context = 'SAN REMO'
label = 'goodcha'
if fid in fid_set:
    matching_rows = df[(df['fid'] == fid) & (df['context'].str.contains(context, regex=False))]
    if not matching_rows.empty:
        row_index = matching_rows.index[0]
        select = df.at[row_index, 'label']
        print(select)
        if select == 'PHI: NULL':
            df.at[row_index, 'label'] = f'{label}: {context}'
        else:
            df.at[row_index, 'label'] = select+f'\\n{label}: {context}'

In [6]:
with open('./AICUP/Second_Phase_Dataset/Second_Phase_Dataset/answer.txt', 'r') as ansfile:
    lines = ansfile.readlines()
fid_set = set(df['fid'].values)
for line in lines:
    splite = line.split('\t')
    fid, label, context = splite[0], splite[1], splite[4]
    context = context.strip()
    if fid in fid_set:
        matching_rows = df[(df['fid'] == fid) & (df['context'].str.contains(context, regex=False))]
        if not matching_rows.empty:
            row_index = matching_rows.index[0]
            select = df.at[row_index, 'label']
            if select == 'PHI: NULL':
                 df.at[row_index, 'label'] = f'{label}: {context}'
            elif select != context:
                 df.at[row_index, 'label'] = select+f'\\n{label}: {context}'

In [7]:
# 將 DataFrame 儲存到 TSV 檔案
df.to_csv(output_file, sep='\t', index=False, header=False, encoding='utf-8')

In [ ]:
import pandas as pd
from pathlib import Path

output_file = 'valid.tsv'
folder_path = '/home/hpds/cheng/bert-eng/AICUP/First_Phase_Release(Correction)/Validation_Release'

def extract_number_from_filename(file_path):
    return int(''.join(filter(str.isdigit, file_path.stem)))

# 創建一個空的 DataFrame
df = pd.DataFrame(columns=['fid', 'idx', 'context'])
data_list = []

file_list = sorted(Path(folder_path).rglob('*.txt'), key=extract_number_from_filename)

for file_path in file_list:
    with open(file_path, 'r', encoding='utf-8') as file:
        char_position = 1
        for idx, line in enumerate(file, start=1):
            if line.strip():
                data_list.append({'fid': file_path.stem, 'idx': char_position, 'context': line.rstrip('\n').replace('\t',' ')})
                char_position += len(line.rstrip('\n'))

df = pd.DataFrame(data_list)
df.reset_index(drop=True, inplace=True)
# 將 DataFrame 儲存到 TSV 檔案
df.to_csv(output_file, sep='\t', index=False, header=False, encoding='utf-8')